# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [125]:
quantity_per_customer = data.groupby(['CustomerID','ProductName', "FirstName", "LastName"])["Quantity"].sum().reset_index()
quantity_per_customer

,CustomerID,ProductName,FirstName,LastName,Quantity
0,33,Apricots - Dried,Lindsay,Santana,1
1,33,Assorted Desserts,Lindsay,Santana,1
2,33,Bandage - Flexible Neon,Lindsay,Santana,1
3,33,"Bar Mix - Pina Colada, 355 Ml",Lindsay,Santana,1
4,33,"Beans - Kidney, Canned",Lindsay,Santana,1
...,...,...,...,...,...
63623,98200,Vol Au Vents,Sammy,Rocha,50
63624,98200,Wasabi Powder,Sammy,Rocha,25
63625,98200,Wine - Fume Blanc Fetzer,Sammy,Rocha,25
63626,98200,Wine - Hardys Bankside Shiraz,Sammy,Rocha,25


In [128]:
quantity_per_customer2 = data.pivot_table(index=['CustomerID', 'ProductID'],
                                        columns=['FirstName', 'LastName'],
                                        values='Quantity',
                                        aggfunc='sum').reset_index()
quantity_per_customer2

FirstName CustomerID ProductID   Aaron               Abel Abigail Adam  \
LastName                       Edwards Washington Hoffman Charles Barr   
0                 33        23     NaN        NaN     NaN     NaN  NaN   
1                 33        51     NaN        NaN     NaN     NaN  NaN   
2                 33        53     NaN        NaN     NaN     NaN  NaN   
3                 33        67     NaN        NaN     NaN     NaN  NaN   
4                 33        70     NaN        NaN     NaN     NaN  NaN   
...              ...       ...     ...        ...     ...     ...  ...   
63623          98200       392     NaN        NaN     NaN     NaN  NaN   
63624          98200       408     NaN        NaN     NaN     NaN  NaN   
63625          98200       418     NaN        NaN     NaN     NaN  NaN   
63626          98200       422     NaN        NaN     NaN     NaN  NaN   
63627          98200       444     NaN        NaN     NaN     NaN  NaN   

FirstName Adriana Aimee Aisha  ... Warren   Wendy  Wesley                \
LastName    Moran  Ware Munoz  ... Robles Wilkins Gardner Hendrix Olsen   
0             NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
1             NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
2             NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
3             NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
4             NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
...           ...   ...   ...  ...    ...     ...     ...     ...   ...   
63623         NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
63624         NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
63625         NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
63626         NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   
63627         NaN   NaN   NaN  ...    NaN     NaN     NaN     NaN   NaN   

FirstName William Xavier Yesenia Yvonne          
LastName    Welch   Owen  Bender   Hale Merritt  
0             NaN    NaN     NaN    NaN     NaN  
1             NaN    NaN     NaN    NaN     NaN  
2             NaN    NaN     NaN    NaN     NaN  
3             NaN    NaN     NaN    NaN     NaN  
4             NaN    NaN     NaN    NaN     NaN  
...           ...    ...     ...    ...     ...  
63623         NaN    NaN     NaN    NaN     NaN  
63624         NaN    NaN     NaN    NaN     NaN  
63625         NaN    NaN     NaN    NaN     NaN  
63626         NaN    NaN     NaN    NaN     NaN  
63627         NaN    NaN     NaN    NaN     NaN  

[63628 rows x 1001 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [62]:
product_by_customer = pd.pivot_table(quantity_per_customer, values="Quantity", index = "ProductName", columns = "CustomerID", fill_value = 0)
product_by_customer

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [64]:
dist_calculation = pdist(X=product_by_customer, metric='euclidean')
len(dist_calculation)

101926

In [65]:
dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

In [69]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=product_by_customer.columns, 
                           columns=product_by_customer.columns)

euclid_dist

"""Esto me hace pensar que la matriz de arriba tendría que haber 
sido con los customers como rows y los productos como columns, 
y no al revés, como pone en el enunciado...
Voy a hacer otra matriz nueva así aquí debajo."""

ValueError: Shape of passed values is (452, 452), indices imply (1000, 1000)

In [70]:
product_by_customer2 = pd.pivot_table(quantity_per_customer, values="Quantity", index = "CustomerID", columns = "ProductName", fill_value = 0)
product_by_customer2

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


In [72]:
dist_calculation = pdist(X=product_by_customer2, metric='euclidean')
len(dist_calculation)

499500

In [73]:
dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [75]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=product_by_customer2.index, 
                           columns=product_by_customer2.index)

euclid_dist

"""Ahora sí tiene sentido, cada customer se relaciona con otro 
y la shape de la matriz concuerda"""


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [76]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(product_by_customer2, 'euclidean'))),
                                index=product_by_customer2.index,
                                columns=product_by_customer2.index)

euclid_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [101]:
#Por ejemplo, los 5 más parecidos al customer 200

top_5_customer = euclid_dist_norm[200].sort_values(ascending=False)[1:6].reset_index()
top_5_customer

,CustomerID,200
0,3317,0.084959
1,1072,0.083972
2,3535,0.082403
3,1920,0.082100
4,3909,0.080920


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [129]:
quantity_per_customer

,CustomerID,ProductName,FirstName,LastName,Quantity
0,33,Apricots - Dried,Lindsay,Santana,1
1,33,Assorted Desserts,Lindsay,Santana,1
2,33,Bandage - Flexible Neon,Lindsay,Santana,1
3,33,"Bar Mix - Pina Colada, 355 Ml",Lindsay,Santana,1
4,33,"Beans - Kidney, Canned",Lindsay,Santana,1
...,...,...,...,...,...
63623,98200,Vol Au Vents,Sammy,Rocha,50
63624,98200,Wasabi Powder,Sammy,Rocha,25
63625,98200,Wine - Fume Blanc Fetzer,Sammy,Rocha,25
63626,98200,Wine - Hardys Bankside Shiraz,Sammy,Rocha,25


In [130]:
records = quantity_per_customer[quantity_per_customer["CustomerID"].isin(top_5_customer["CustomerID"])]
records

,CustomerID,ProductName,FirstName,LastName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",Guillermo,Leach,1
1083,1072,Appetizer - Sausage Rolls,Guillermo,Leach,1
1084,1072,"Beans - Kidney, Canned",Guillermo,Leach,1
1085,1072,Beef - Montreal Smoked Brisket,Guillermo,Leach,1
1086,1072,Beef - Striploin Aa,Guillermo,Leach,1
...,...,...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,Keri,Stafford,2
3264,3909,Wine - Fume Blanc Fetzer,Keri,Stafford,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",Keri,Stafford,1
3266,3909,Wine - Toasted Head,Keri,Stafford,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [138]:
records_ranked = records.groupby("ProductName")["Quantity"].sum().reset_index()
records_ranked = records_ranked.sort_values(by="Quantity", ascending=False)
records_ranked

,ProductName,Quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3
...,...,...
86,Kiwi,1
87,Knife Plastic - White,1
90,Lambcasing,1
91,"Lentils - Red, Dry",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [178]:
#Vuelvo a hacer el ejemplo con el customer 200
merge = pd.merge(records_ranked, product_by_customer, on='ProductName', how='inner')
merge

,ProductName,Quantity,33,200,264,356,412,464,477,639,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,"Pepper - Black, Whole",3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Knife Plastic - White,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0
2,Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
3,Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Kiwi,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,"Lentils - Red, Dry",1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,Lettuce - California Mix,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,Longos - Grilled Chicken With,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0


In [151]:
top_not_purchased = merge.loc[merge[200] == 0].sort_values(by="Quantity", ascending =False)
top_not_purchased_filtered = top_not_purchased[[200, "ProductName", "Quantity"]]
top_not_purchased_filtered[1:6]

,200,ProductName,Quantity
2,0.0,Bay Leaf,3
3,0.0,Pork - Kidney,3
6,0.0,Wanton Wrap,3
1,0.0,Muffin - Carrot Individual Wrap,3
47,0.0,Wine - Blue Nun Qualitatswein,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [170]:
recommendations = {}
customer_ids = list(data.CustomerID.unique())
for c in customer_ids:
    top_5_customer = euclid_dist_norm[c].sort_values(ascending=False)[1:6].reset_index()
    records = quantity_per_customer[quantity_per_customer["CustomerID"].isin(top_5_customer["CustomerID"])]
    records_ranked = records.groupby("ProductName")["Quantity"].sum().reset_index()
    records_ranked = records_ranked.sort_values(by="Quantity", ascending=False)
    merge = pd.merge(records_ranked, product_by_customer, on='ProductName', how='inner')
    top_not_purchased = merge.loc[merge[c] == 0].sort_values(by="Quantity", ascending =False)
    top_not_purchased_filtered = top_not_purchased[[c, "ProductName", "Quantity"]]
    recommendations[c] = list(top_not_purchased_filtered["ProductName"][1:6])
recommendations

{61288: ['Chicken - Soup Base',
  'Lime Cordial - Roses',
  'Macaroons - Two Bite Choc',
  'Flavouring - Orange',
  'Water - Mineral, Natural'],
 77352: ['Tahini Paste',
  'Tuna - Salad Premix',
  'Chicken - Soup Base',
  'Isomalt',
  'Wiberg Super Cure'],
 40094: ['Puree - Mocha',
  'Tuna - Salad Premix',
  'Sherry - Dry',
  'Beer - Sleemans Cream Ale',
  'Chocolate - Feathers'],
 23548: ['Wanton Wrap',
  'Puree - Mocha',
  'Pernod',
  'Sprouts - Baby Pea Tendrils',
  'Banana Turning'],
 78981: ['Lettuce - Frisee',
  'Yogurt - Blueberry, 175 Gr',
  'Cinnamon Buns Sticky',
  'Pop Shoppe Cream Soda',
  'Beef - Inside Round'],
 83106: ['Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Cheese - Mix',
  'Sauce - Hollandaise',
  'Chocolate - Dark'],
 11253: ['Pomello',
  'Tomatoes Tear Drop',
  'Skirt - 29 Foot',
  'Wine - Blue Nun Qualitatswein',
  'Mayonnaise - Individual Pkg'],
 35107: ['Bandage - Flexible Neon',
  'Lentils - Red, Dry',
  'Thermometer Digital',
  'Wine - Hardys

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [176]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.T

,0,1,2,3,4
61288,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange,"Water - Mineral, Natural"
77352,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt,Wiberg Super Cure
40094,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale,Chocolate - Feathers
23548,Wanton Wrap,Puree - Mocha,Pernod,Sprouts - Baby Pea Tendrils,Banana Turning
78981,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda,Beef - Inside Round
...,...,...,...,...,...
49005,Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab",Beets - Mini Golden
41286,Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style,Wasabi Powder,"Placemat - Scallop, White"
85878,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine,Meldea Green Tea Liquor
68506,Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed",Cup - Translucent 7 Oz Clear


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [179]:
cosine_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(product_by_customer2, 'cosine'))),
                                index=product_by_customer2.index,
                                columns=product_by_customer2.index)

cosine_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [180]:
recommendations = {}
customer_ids = list(data.CustomerID.unique())
for c in customer_ids:
    top_5_customer = cosine_dist_norm[c].sort_values(ascending=False)[1:6].reset_index()
    records = quantity_per_customer[quantity_per_customer["CustomerID"].isin(top_5_customer["CustomerID"])]
    records_ranked = records.groupby("ProductName")["Quantity"].sum().reset_index()
    records_ranked = records_ranked.sort_values(by="Quantity", ascending=False)
    merge = pd.merge(records_ranked, product_by_customer, on='ProductName', how='inner')
    top_not_purchased = merge.loc[merge[c] == 0].sort_values(by="Quantity", ascending =False)
    top_not_purchased_filtered = top_not_purchased[[c, "ProductName", "Quantity"]]
    recommendations[c] = list(top_not_purchased_filtered["ProductName"][1:6])
recommendations

{61288: ['Wine - Two Oceans Cabernet',
  'Tofu - Firm',
  'Appetizer - Sausage Rolls',
  'Dc - Frozen Momji',
  'Snapple Lemon Tea'],
 77352: ['Fondant - Icing',
  'Lettuce - Frisee',
  'Pork - Kidney',
  'Bread - French Baquette',
  'Chicken - Leg, Boneless'],
 40094: ['Beef - Inside Round',
  'Bread - Roll, Soft White Round',
  'Truffle Cups - Brown',
  'Cake - Mini Cheesecake',
  'Anchovy Paste - 56 G Tube'],
 23548: ['Bagel - Plain',
  'Olives - Stuffed',
  'Veal - Sweetbread',
  'Cinnamon Buns Sticky',
  'Barramundi'],
 78981: ['Cookies - Assorted',
  'Cheese - Cottage Cheese',
  'Baking Powder',
  'Quiche Assorted',
  'Tart Shells - Sweet, 4'],
 83106: ['Onions - Vidalia',
  'Scallops - Live In Shell',
  'Pastry - Choclate Baked',
  'Soup - Canadian Pea, Dry Mix',
  'Bread - Italian Roll With Herbs'],
 11253: ['Beef - Tenderlion, Center Cut',
  'Veal - Eye Of Round',
  'Soup - Canadian Pea, Dry Mix',
  'Wine - Gato Negro Cabernet',
  'Onions - Cippolini'],
 35107: ['Wine - Cahors

In [181]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.T

,0,1,2,3,4
61288,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji,Snapple Lemon Tea
77352,Fondant - Icing,Lettuce - Frisee,Pork - Kidney,Bread - French Baquette,"Chicken - Leg, Boneless"
40094,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake,Anchovy Paste - 56 G Tube
23548,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky,Barramundi
78981,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted,"Tart Shells - Sweet, 4"
...,...,...,...,...,...
49005,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi,Milk - 1%
41286,"Salsify, Organic",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves,Wine - Alsace Gewurztraminer
85878,French Pastry - Mini Chocolate,Cheese - Parmesan Grated,Foam Dinner Plate,Cheese - Mix,Wine - Two Oceans Cabernet
68506,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing,Garbag Bags - Black


In [ ]:
#There are significant changes between the euclidean and the cosine metrics.